In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
url = 'https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/locations.csv?raw=true'
df = pd.read_csv(url)

In [ ]:
df['location'] = df['location'].str.lower()

In [ ]:
Pfizer_country = df[df['vaccines'].str.contains('Pfizer/BioNTech')].drop(['source_website','source_name'],axis=1)
Moderna_country = df[df['vaccines'].str.contains('Moderna')].drop(['source_website','source_name'],axis=1)

In [ ]:
Pfizer_country['location'] = Pfizer_country['location'].str.lower()
Moderna_country['location'] = Moderna_country['location'].str.lower()

In [ ]:
Pfizer_country.to_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/Pfizer_country.csv')
Moderna_country.to_csv('/content/drive/MyDrive/DA팀 /프로젝트/코로나 백신/Moderna_country.csv')

In [ ]:
def Preprocessing(file):
  raw_data = file
  raw_data = raw_data.drop(['id','user_created','source','is_retweet'],axis=1)
  data1 = raw_data.drop(['text','user_descrption','user_friends','retweets'],axis=1)
  data2 = pd.DataFrame(raw_data['text'])
  #location
  data_co = data1.copy()
  for x in Pfizer_country['location']:
    for k in range(len(data_co)):
      if x in str(data_co['user_location'][k]):
        data_co['user_location'][k] = 1
  for k in range(len(data1)):
    if 'london' in str(data1['user_location'][k]):
      data_co['user_location'][k] = 1
  for k in range(len(data1)):
    if 'uk' in str(data1['user_location'][k]):
      data_co['user_location'][k] = 1
  data1 = data_co
  #normalization
  data1['user_followers']=data1['user_followers']/max(data1['user_followers'])
  data1['user_favourites']=data1['user_favourites']/max(data1['user_favourites'])
  data1['favorites']=data1['favorites']/max(data1['favorites'])
  #date처리
  data1['date'] = data1['date'].str[:10]
  Year=[]
  month=[]
  day=[]
  for x in range(len(data1)):
    a = datetime.strptime(data1['date'][x],"%Y-%m-%d")
    Year.append(a.year)
    month.append(a.month)
    day.append(a.day)
  data1['Year']=Year
  data1['month']=month
  data1['day']=day
  data1 = data1.drop('date',axis=1)
  #hashtags Word2Vec









    data_co['user_location']=data['user_location'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
